In [1]:
import numpy as np
from numpy import array
from pickle import dump, load
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.layers import Dense, LSTM, Embedding
from keras.optimizers import Adadelta
from keras.callbacks import EarlyStopping

from txt2sequence import convert_to_sequences

Using TensorFlow backend.


# Convert Text to Sequences
N.B. If 'texts/hp_sequences.txt' has already been generated you don't need to run the cell below

In [2]:
convert_to_sequences()

importing books
cleaning text
removing infrequent words
saving sequences to file


# Train Language Model

In [3]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# load
in_filename = 'texts/hp_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [5]:
vocab_size = len(tokenizer.word_index) + 1
sequences = array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [6]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=seq_length))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(128, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 64)            528832    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 8263)              1065927   
Total params: 1,841,671
Trainable params: 1,841,671
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
# compile model
optimizer = Adadelta()
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min', restore_best_weights=True)

In [8]:
# fit model
model.fit(X, y, batch_size=1024, epochs=1000, callbacks=[earlyStopping], validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Train on 1058975 samples, validate on 117664 samples
Epoch 1/1000
1058975/1058975 [==============================] - 1849s 2ms/step - loss: 6.4580 - accuracy: 0.0706 - val_loss: 6.3575 - val_accuracy: 0.0727
Epoch 2/1000
1058975/1058975 [==============================] - 1842s 2ms/step - loss: 6.3685 - accuracy: 0.0732 - val_loss: 6.3612 - val_accuracy: 0.0727
Epoch 3/1000
1058975/1058975 [==============================] - 1871s 2ms/step - loss: 6.2548 - accuracy: 0.0772 - val_loss: 6.1066 - val_accuracy: 0.0857
Epoch 4/1000
1058975/1058975 [==============================] - 2004s 2ms/step - loss: 5.9305 - accuracy: 0.1014 - val_loss: 5.8148 - val_accuracy: 0.1092
Epoch 5/1000
1058975/1058975 [==============================] - 2043s 2ms/step - loss: 5.6527 - accuracy: 0.1254 - val_loss: 5.6514 - val_accuracy: 0.1227
Epoch 6/1000
1058975/1058975 [==============================] - 1953s 2ms/step - loss: 5.4645 - accuracy: 0.1410 - val_loss:

In [14]:
model.save('models/080520/model.h5')
with open('models/080520/model.json', 'w') as json_file:
    json_file.write(model.to_json())
# save the tokenizer
dump(tokenizer, open('models/080520/tokenizer.pkl', 'wb'))